In [13]:
import re
sictab = {}
linetab = {}

def getbestfit(sicword,curcit):
    bestfit = 0
    bestword= ''
    workl = re.sub('<[^>]+>',' ',linetab[curcit])
    workl = re.sub('·','',workl)
    workl = re.sub('[\.,;]','',workl)
    for word in workl.split():
        curfit = fuzz.ratio(word,sicword)
        if(curfit > bestfit):
            bestfit = curfit
            bestword = word
    return(bestword)
    
def dogrc(fname):
    global linetab
    global sictab
    innotes = 0
    outfname = re.sub('1\.xml','2a.xml',fname)
    f = open(fname)
    curlnum = 0
    outf = open(outfname,'w')
    for l in f:
        l = re.sub('\s+$','',l)
        l = re.sub('^[0-9]+$','',l)
        if(not l):
            continue


        
        if(re.search('<pb',l)):
            if(innotes):
                print('</note>',file=outf)
                
            innotes = 0
            print(l,file=outf)
            continue
        if(re.search('<note/>',l)):
            innotes = 1
            print('<note>',file=outf)
            continue

        if(innotes):
            l = re.sub('^([0-9\-]+)\.\s+(.+)','<ref target="\g<1>">\g<2></ref>',l)
            l = re.sub('(<ref[^>]+>)([^A-Za-z]+)\s+([A-Za-z])','\g<1><sic>\g<2></sic> \g<3>',l)
            m = re.search('<ref target="([0-9]+)[^>]+><sic>([^<]+)',l)
            if(m):
                curcit = m[1]
                cursic = m[2]
                #print(curcit,cursic)
                bestfit = getbestfit(cursic,curcit)
                if(not curcit in sictab):
                    sictab[curcit] = [cursic]
                else:
                    sictab[curcit].append(cursic)
                l = re.sub('<sic>','<lem>'+bestfit+'</lem> <sic>',l)
            print(l,file=outf)
            continue
        l = re.sub('(.+)\s+([0-9]+)$','<l n="\g<2>">\g<1></l>',l)
        l = re.sub('(ἀντ\.|στρ\.)([^<]*)','<milestone unit="strophe" n="\g<1>\g<2>"/>',l)
        m = re.search('<l n="([0-9]+)">(.+)</l>',l)
        if(m):
            tmplnum = int(m[1])
            linetab[m[1]] = m[2]
            if(not tmplnum == curlnum + 1):
                print('jump',curlnum,tmplnum,l)
            curlnum = tmplnum
            #print(l,file=outf)
            #continue
        if(re.search('<',l) and not re.search('<(mile|corr|l|gap|delἶ)',l)):
            print(l,file=outf)
            continue
        l = re.sub('\*\s*([^,\.\s;<]+)','<corr>\g<1></corr>',l)
        l = re.sub('·</lem>','</lem>·',l)

        if(not re.search('<l ',l)):
            curlnum = curlnum + 1
            
            print('<l n="'+str(curlnum)+'">'+l+'</l>',file=outf)
            linetab[str(curlnum)] = l
        else:
            
            print(l,file=outf)
        
    print(fname,outfname)
    

dogrc('aesch.eum.sidgwick-grc1.xml')

jump 158 160 <l n="160">πάρεστι μαστίκτορος δαΐου </l>
jump 168 170 <l n="170">μυχὸν ἔχρανας αὐτόσσυτος, αὐτόκλητος,</l>
jump 173 175 <l n="175">ὑπό τε γᾶν φυγὼν οὔ ποτ᾽ ἐλευθεροῦται·</l>
jump 177 179 <l n="179">ἔξω, κελεύω, τῶνδε δωμάτων τάχος</l>
jump 253 255 <l n="255">ὅρα, ὅρα μάλ᾽ αὖ, λεῦσσέ * τε πάντα μὴ</l>
jump 258 260 <l n="260">ὑπόδικος θέλει γενέσθαι χερῶν.</l>
jump 303 305 <l n="305">καὶ ζῶν με δαίσεις, οὐδὲ πρὸς βωμῷ σφαγείς·</l>
jump 347 350 <l n="350">ἀθανάτων δ᾽ ἀπέχειν χέρας, οὐδέ τις ἐστὶ</l>
jump 353 355 <l n="355">ἀνατροπάς, ὅταν Ἄης</l>
jump 363 366 <l n="366">ὡς ἀπηξιώσατο, -</l>
jump 373 375 <l n="375">ἁμετέραις ἐφόδοις μελανείμοσιν, ὀρχησ-</l>
jump 528 530 <l n="530">παντὶ μέσῳ τὸ κράτος θεὸς ὤπασεν·</l>
jump 553 555 <l n="555">βιαίως ξὺν χρόνῳ καθήσειν,</l>
jump 792 794 <l n="794">ἐμοὶ πίθεσθε μὴ βαρυστόνως φέρειν</l>
jump 823 825 <l n="825">θεαὶ βροτῶν στήσητε δύσκηλον χθόνα.</l>
jump 848 850 <l n="850">φρονεῖν δὲ κἀμοὶ Ζεὺς ἔδωκεν οὐ κακῶς.</l>
jump 878 880 <

In [6]:
from fuzzywuzzy import fuzz

for foo in sictab:
    if(not foo in linetab):
        print()
        print(foo,sictab[foo],'noline')
        continue

    bestfit = 0
    workl = re.sub('<[^>]+>',' ',linetab[foo])
    workl = re.sub('·','',workl)
    workl = re.sub('[\.,;]','',workl)
    for word in workl.split():
        curfit = fuzz.ratio(word,sictab[foo][0])
        if(curfit > bestfit):
            bestfit = curfit
            bestword = word
    print(bestfit,foo,sictab[foo][0],bestword,workl,sep='\t')
        
    #print(foo,sictab[foo],linetab[foo])

86	18	χρόνοις	θρόνοις	ἵζει τέταρτον τόνδε μάντιν ἐν  θρόνοις 
89	23	ἀναστροφά	ἀναστροφή	κοίλη φίλορνις δαιμόνων ἀναστροφή
73	24	δ᾽ ἔχει	ἔχει	Βρόμιος ἔχει τὸν χῶρον οὐδ᾽ ἀμνημονῶ
59	27	πλείστους	Πλειστοῦ	 Πλειστοῦ  τε πηγάς καὶ Ποσειδῶνος κράτος
75	32	ἴτην	ἴτων	ἴτων πάλῳ λαχόντες ὡς νομίζεται
73	36	στάσιν	βάσιν	ὡς μήτε σωκεῖν μήτε μ᾿ ἀκταίνειν  βάσιν 
86	40	θεομισή	θεομυσή	ὁρῶ δ᾽ ἐπ᾿ ὀμφαλῷ μὲν ἄνδρα θεομυσή
80	41	ἕδρας	ἕδραν	ἕδραν ἔχοντα προστρόπαιον αἵματι
89	42	νεοσπαθὲς	νεοσπαδὲς	στάζοντα χεῖρας καὶ νεοσπαδὲς ξίφος
29	46	δέ τ'	δὲ	πρόσθεν δὲ τἀνδρὸς τοῦδε θαυμαστὸς λόχος
94	53	πλαστοῖσι	πλατοῖσι	ῥέγκουσι δ᾽ οὐ  πλατοῖσι  φυσιάμασιν
57	54	δία	λίβα	ἐκ δ᾽ ὀμμάτων λείβουσι δυσφιλῆ  λίβα 
40	58	εἴ τις γαῖα	ἥτις	οὐδ᾽ ἥτις αἷα τοῦτ᾽ ἐπεύχεται γένος
80	59	πόνων	πόνον	τρέφουσ᾽ ἀνατὶ μὴ μεταστένειν πόνον
86	71	ἐπὶ	ἐπεὶ	κακῶν δ᾽ ἕκατι κἀγένοντ᾽ ἐπεὶ κακὸν
47	73	μίσημά τ'	μισήματ᾿	μισήματ᾿ ἀνδρῶν καὶ θεῶν Ὀλυμπίων
71	76	βεβῶντ᾽ ἂν	βιβῶντ᾽	 βιβῶντ᾽  ἀν᾿ ἀεὶ τὴν πλανοστιβῆ χθόνα
83	77	πόντου	πόντο